LangChain의 RAG 콤포넌트 - 텍스트 분할기(Text Splitter)

학습목표
1. 텍스트 분할의 필요성과 RAG 시스템에서의 역할 이해
2. 다양한 텍스트 분할 전략(문자기반, 재귀, 토큰기반, 의미기반) 비교
3. 각 분할기의 파라미터(chunk_size, chunk_overlap, separator 등) 이해 및 활용
4. 실무에서 상황에 맞는 적절한 분할 전략 선택 능력 배양

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

In [1]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('../data/transformer.pdf', mode="single")

# 동기 로딩
pdf_docs = pdf_loader.load()
pdf_docs

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/transformer.pdf', 'total_pages': 15}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nI

텍스트 분할 Text Splitting
- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항
	1. 문서의 구조와 형식
	2. 원하는 청크 크기
	3. 문맥 보존의 중요도
	4. 처리 속도

In [8]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')
print('Document 객체 출력(첫번째 분할된 텍스트로 생성) : ', pdf_docs[0])

첫 번째 문서의 텍스트 길이: 39643
Document 객체 출력(첫번째 분할된 텍스트로 생성) :  page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attent

1. CharacterTextSplitter
- 가장 기본적인 분할 방식
- 문자 수를 기준으로 텍스트를 분할
- 단순하지만 문맥을 고려하지 않는다는 단점이 있음

- CharaterTextSplitter 주요 파라미터
	- separator : `\n\n` > 청크를 구분하는 구분자
	- chunk_size : `1000` > 각 청크의 최대 길이 (문자 수)
	- chunk_overlap : `200` > 인접 청크 간 중복되는 문자 수 (문맥 보존)
	- length_function : `len` > 텍스트 길이를 계산하는 함수
	- is_separator_regex : `False` > separator가 정규식표현인지 여부

In [10]:
from langchain_text_splitters import CharacterTextSplitter

# 텍스트 분할기 초기화 - 기본 설정값 적용
text_splitter = CharacterTextSplitter(
	# CharatrerTextSplitter 기본 설정값
	separator="\n\n",					# 청크 구분자 : 두개의 개행 문자
	is_separator_regex=False,	# 구분자가 정규식인지 여부

	# TextSplitter의 기본 설정값
	chunk_size = 1000,				# 청크 길이
	chunk_overlap = 200,			# 청크 중첩
	length_function = len,		# 길이 함수 (문자열 길이)
	keep_separator = False,		# 구분자 유지 여부
	add_start_index = False,	# 시작 인덱스 추가 여부
	strip_whitespace = True,	# 공백 제거 여부
)

# print('long_text : ', long_text)
# 텍스트 분할 - split_text() 매서드 사용
text = text_splitter.split_text(long_text)

# print(f'분할된 텍스트 개수 : {len(text)}')
# print(f'첫 번째 분할된 텍스트 : {text[0]}')
print('Document 객체 출력(첫번째 분할된 텍스트로 생성) : ', pdf_docs[0])

Document 객체 출력(첫번째 분할된 텍스트로 생성) :  page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispens

In [13]:
from langchain_text_splitters import CharacterTextSplitter

# 문장 구분자를 개행문자로 설정
text_splitter = CharacterTextSplitter(
	separator = " ",		# 청크 구분자: 개행문자
	chunk_size = 1000, 	# 청크 길이
	chunk_overlap = 200 # 청크 중첩
)

# split_documents() 매서드 사용 : Document 객체를 여러개의 작은 청크로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])

# 분할 된 텍스트 개수 출력
print(f'분할된 텍스트 개수 : {len(chunks)}')

# 각 청크의 텍스트 길이 출력
# for i, chunk in enumerate(chunks):
# 	print(f'청크 {i+1}의 텍스트 길이 : {len(chunk.page_content)}')

# 첫 번째 청크의 텍스트 출력
# print(f'첫 번째 청크의 텍스트: {chunks[0].page_content}')
print(f'두 번째 청크의 텍스트: {chunks[1].page_content}')

분할된 텍스트 개수 : 50
두 번째 청크의 텍스트: and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
∗Equal contribution. Listi

2. RecursiveCharacterTextSplitter
- 재귀적으로 텍스트를 분할
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 분할
- 문맥을 더 잘 보존할 수 있음

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
	chunk_size = 1000,
	chunk_overlap = 200,
	length_function = len,
	separators = ['\n\n', '\n', r'(?<=[.!?])\s+'] # 구분자 - 재귀적으로 순차적으로 적용
)

# split_documents() 메서드 사용 : Document 객체를 여러개의 작은 청크 분서로 분할
chunk = text_splitter.split_documents(pdf_docs)
print(f'생성된 텍스트의 청크 수 : {len(chunks)}')
print(f'각 청크의 길이 : {list(len(chunk.page_content) for chunk in chunks)}')

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunk[:5]:
	print(chunk.page_content[:200])
	print("-" * 100)
	print(chunk.page_content[-200:])
	print("=" * 100)
	print()


생성된 텍스트의 청크 수 : 50
각 청크의 길이 : [997, 995, 993, 988, 1000, 995, 991, 1000, 999, 1000, 987, 999, 998, 993, 998, 992, 999, 996, 1000, 996, 998, 991, 997, 996, 999, 987, 990, 997, 995, 998, 997, 990, 992, 997, 999, 997, 998, 993, 998, 998, 995, 999, 991, 986, 1000, 1000, 995, 996, 993, 265]
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need

----------------------------------------------------------------------------------------------------
the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions

mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine transla
--------

4. 토큰 수를 기반으로 분할

	(1) tiktoken
		- LangChain OpenAI에서 만든 BPE Tokenizer

In [22]:
len(pdf_docs[0].page_content)

from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
	model_name="gpt-4.1",
	chunk_size=300,
	chunk_overlap=0
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
	print(chunk.page_content[:200])
	print("-" * 100)
	print(chunk.page_content[-200:])
	print("=" * 100)
	print()

생성된 청크 수: 36
각 청크의 길이: [1146, 1374, 1376, 1503, 1425, 1217, 1357, 1307, 1229, 1182, 1333, 1147, 1204, 1423, 1323, 1253, 959, 678, 1087, 1225, 582, 918, 1368, 737, 1344, 1029, 835, 995, 933, 947, 954, 834, 993, 879, 770, 740]
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need

----------------------------------------------------------------------------------------------------
spensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly

less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 Eng
----------------------------------------------------------------------

In [26]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-4.1")

for chunk in chunks[:5]:
		# 각 청크를 토큰화
		tokens = tokenizer.encode(chunk.page_content)
		print(f'각 청크의 단어 수 확인 : {len(tokens)}')
		print(f'각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력) : {tokens[:10]}')
		print('토큰 ID를 실제 토큰(문자열)로 변환해서 출력')
		token_strings = [tokenizer.decode([token]) for token in tokens[:10]]
		print(token_strings)

		print("=" * 50)


각 청크의 단어 수 확인 : 280
각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력) : [110436, 7937, 118839, 382, 5181, 11, 5800, 43378, 36800, 14158]
토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['Provided', ' proper', ' attribution', ' is', ' provided', ',', ' Google', ' hereby', ' grants', ' permission']
각 청크의 단어 수 확인 : 289
각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력) : [2695, 1058, 316, 8513, 13, 5339, 2359, 136969, 220, 2029]
토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['less', ' time', ' to', ' train', '.', ' Our', ' model', ' achieves', ' ', '28']
각 청크의 단어 수 확인 : 286
각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력) : [105849, 289, 33686, 17, 102370, 11, 39866, 1039, 11965, 3490]
토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['implement', 'ing', ' tensor', '2', 'tensor', ',', ' replacing', ' our', ' earlier', ' code']
각 청크의 단어 수 확인 : 289
각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력) : [639, 47913, 723, 1398, 2155, 2049, 1217, 23053, 2359, 6198]
토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['com', 'putation', ' [', '32', '],', ' while', ' also', ' improving', ' model', ' performance']
각 청크의 단어 수 확인 : 286
각 청크의 토큰화 결과 확인 (

In [28]:
chunk.metadata
print(chunk.page_content)

reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and

	(2) Hugging Face 토크나이저
		- Hugging Face Tokenizer 모델의 토큰수를 기준으로 분할

In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

tokenizer

XLMRobertaTokenizerFast(name_or_path='BAAI/bge-m3', vocab_size=250002, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [31]:
tokens = tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

# 토큰을 출력 (토큰 ID를 실제 토큰(문자열)로 변환)
print(tokenizer.convert_ids_to_tokens(tokens))

[0, 107687, 5, 20451, 54272, 16367, 5, 2]
['<s>', '▁안녕하세요', '.', '▁반', '갑', '습니다', '.', '</s>']


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# Huggingface 토크나이저를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
	tokenizer = tokenizer,
	chunk_size = 300,
	chunk_overlap = 0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]]) # 첫번째 문서만 분할


print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
	# 각 청크를 토큰화
	tokens = tokenizer.encode(chunk.page_content)
	print(f"각 청크의 단어 수 확인: {len(tokens)}")
	print(f"각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력): {tokens[:10]}")
	print()
	print("토큰 ID를 실제 토큰(문자열)로 변환해서 출력")
	token_strings = tokenizer.convert_ids_to_tokens(tokens[:10])
	print(token_strings)
	
	print("=" * 50)
	print()

생성된 청크 수: 36
각 청크의 길이: [1220, 1300, 1273, 1303, 1232, 1081, 1030, 1162, 1222, 1130, 1064, 1131, 1047, 1078, 1232, 1302, 1174, 1109, 909, 1059, 1224, 1002, 1069, 1169, 979, 1196, 1029, 944, 1027, 1001, 1045, 925, 968, 1149, 1138, 684]

각 청크의 단어 수 확인: 299
각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력): [0, 123089, 71, 27798, 99, 179236, 83, 62952, 4, 1815]

토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['<s>', '▁Provide', 'd', '▁proper', '▁at', 'tribution', '▁is', '▁provided', ',', '▁Google']

각 청크의 단어 수 확인: 295
각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력): [0, 47, 9, 191697, 153648, 66211, 4, 224588, 645, 70]

토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['<s>', '▁to', '-', 'German', '▁translation', '▁task', ',', '▁improving', '▁over', '▁the']

각 청크의 단어 수 확인: 302
각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력): [0, 29479, 214, 1492, 4970, 304, 510, 4970, 4, 456]

토큰 ID를 실제 토큰(문자열)로 변환해서 출력
['<s>', '▁implement', 'ing', '▁ten', 'sor', '2', 'ten', 'sor', ',', '▁re']

각 청크의 단어 수 확인: 287
각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력): [0, 88551, 136912, 7, 23, 181135, 43315, 227066, 8305, 